# Searching with ONE
ONE contains a method that allows you to search for sessions of interest. The possible search terms can be listed using,

In [1]:
from one.api import ONE
one = ONE(base_url='https://openalyx.internationalbrainlab.org')

print(one.search_terms())

('dataset', 'date_range', 'laboratory', 'number', 'project', 'subject', 'task_protocol')


We can search for sessions within a specified date range (inclusive)

In [2]:
from pprint import pprint
eids = one.search(date_range=['2020-01-01', '2021-01-01'])
pprint(eids)

['4ecb5d24-f5cc-402c-be28-9d0f7cb14b3a',
 'c7bd79c9-c47e-4ea5-aea3-74dda991b48e',
 '4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b']


A single date can be provided instead of a range. Also to define only the upper or lower date bound, set the other element to None.

In [ ]:
assert one.search(date_range=['2020-01-01', '2021-01-01']) == one.search(date_range='2021-01-01')

To get more information about the sessions we can add a details=True flag

In [3]:
eids, details = one.search(date_range=['2020-01-01', '2021-01-01'], details=True)
pprint(details)

(['4ecb5d24-f5cc-402c-be28-9d0f7cb14b3a',
  'c7bd79c9-c47e-4ea5-aea3-74dda991b48e',
  '4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b'],
 [{'date': datetime.date(2020, 9, 21),
   'lab': 'hoferlab',
   'number': 1,
   'project': 'ibl_neuropixel_brainwide_01',
   'subject': 'SWC_043',
   'task_protocol': '_iblrig_tasks_ephysChoiceWorld6.4.2'},
  {'date': datetime.date(2020, 9, 19),
   'lab': 'zadorlab',
   'number': 1,
   'project': 'ibl_neuropixel_brainwide_01',
   'subject': 'CSH_ZAD_029',
   'task_protocol': '_iblrig_tasks_ephysChoiceWorld6.4.2'},
  {'date': datetime.date(2020, 1, 8),
   'lab': 'churchlandlab',
   'number': 1,
   'project': 'ibl_neuropixel_brainwide_01',
   'subject': 'CSHL049',
   'task_protocol': '_iblrig_tasks_ephysChoiceWorld6.2.5'}])


Multiple search terms can also be combined, for example we can search for any sessions from the subject SWC_043 that contain the datasets spikes.times and spikes.clusters

In [4]:
eids = one.search(subject='SWC_043', dataset=['spikes.times', 'spikes.clusters'])
pprint(eids)

['4ecb5d24-f5cc-402c-be28-9d0f7cb14b3a']


More search terms are available when making [remote queries](../one_modes.html) (using the remote
Alyx database instead of the local cache).  You can view all the remote search terms with the
'remote' arg:

In [ ]:
one.search_terms('remote')
eids = one.search(performance_gte=70, query_type='remote')

<div class="alert alert-warning">
Warning.

Remote search queries behave slightly differently. See "<a href="#Gochas">Gochas</a>" below.
</div>

Search term arguments may be shortened, so long as they are not ambiguous:

In [ ]:
assert one.search(task_protocol='training') == one.search(task='training')
assert one.search(project='brainwide') == one.search(proj='brainwide')
assert one.search(date_range='2021-01-01') == one.search(date='2021-01-01')

`one.search(dat='2020-01-01')` will raise a ValueError as 'dat' could be short for both 'date_range' and 'dataset'.

<div class="alert alert-warning">
Warning.

There are more search terms when using remote mode, for example 'data' can match both 'dataset' and 'datasets' in remote mode.
</div>


To find out more information about the `one.search` method we can use the help function

In [5]:
help(one.search)

Help on method search in module one.api:

search(details=False, query_type=None, **kwargs) method of one.api.OneAlyx instance
    Searches sessions matching the given criteria and returns a list of matching eids
    
    For a list of search terms, use the method
    
     one.search_terms(query_type='remote')
    
    For all of the search parameters, a single value or list may be provided.  For dataset,
    the sessions returned will contain all listed datasets.  For the other parameters,
    the session must contain at least one of the entries. NB: Wildcards are not permitted,
    however if wildcards property is False, regular expressions may be used for all but
    number and date_range.
    
    Parameters
    ----------
    dataset : str, list
        list of dataset names. Returns sessions containing all these datasets.
        A dataset matches if it contains the search string e.g. 'wheel.position' matches
        '_ibl_wheel.position.npy'
    date_range : str, list, datetime.

## Advanced searching
By default ONE searches most terms function as an LIKE OR expression, returning results that contain any of
the search values as a substring.  For example `one.search(subject=['foo', 'bar'])` returns all sessions where
the subject name contains 'foo' or contains 'bar'. The exception is the dataset search term, which is a LIKE AND
expression, i.e. the session must contain one or more dataset names containing 'foo' AND one or more datasets
containing 'bar'.  Note that all expressions are case-sensitive in auto/local mode and case-insensitive in remote mode.

For more precise searches, regular expressions (a.k.a. regex) can be used.  This is on by default and can be
deactivated by setting the wildcards flag: `one.wildcards = False` (note that this also affects the list and
load methods, see [the advanced loading section of Loading with ONE](../one_load/one_load.html#Advanced-loading)
for more details).

Regex allows one to make exact searches by asserting the start and end of the string:

In [ ]:
eids = one.search(subject='FD_04')  # includes sessions with subject 'MFD_04'
assert one.wildcards is True, 'the wildcards flag must be True for regex expressions'
eids = one.search(subject='^FD_04$')  # exact subject name match

Likewise, to search for sessions that include one dataset OR another, we can use the `|` character in
our regex:

In [ ]:
# Sessions containing either leftCamera.times OR rightCamera.times:
eids = one.search(proj='brainwide', dataset='leftCamera\.times|rightCamera\.times')
# XOR expressions are also possible:
eids = one.search(proj='brainwide', dataset='(leftCamera\.times|rightCamera\.times){1}')

Note that the wildcards flag causes certain characters to be interpreted differently (e.g. `.` matches
any character).  To avoid this, either set the wildcards flag to False or escape the string using `re.escape`:

In [ ]:
import re
subject = 'NYU-14.1'
if one.wildcards:
    subject = re.escape(subject)
eids = one.search(subject=subject)  # 'NYU\\-14\\.1'

### Gotchas
ONE.search strikes a balance between usability, functionality, and stability. We have been careful
to ensure that the results of the search function have remained consistent across versions, however
there are some confusing and unintuitive behaviours as a result...

#### Difference between search term behaviours
As mentioned above, different search terms perform differently. Below are the search terms and their
approximate SQL equivalents:

| Term         | Lookup    |
|--------------|-----------|
| dataset      | LIKE AND  |
| number       | EXACT     |
| date_range   | BETWEEN   |
| subject, etc.| LIKE OR   |

Combinations of terms form a logical AND, for example `one.search(subject=['foo', 'bar'], project='baz')`
searches for sessions where the subject name contains foo OR bar, AND the project contains baz.

#### Difference between remote mode search terms
Many search terms perform differently between auto/local mode and [remote mode](../one_modes.html),
namely in remote mode, search queries are case-insensitive.

#### The dataset, datasets and dataset_types remote arguments
In remote mode there are three ways to search for datasets:

* **dataset** - a partial, case-insensitive match of a single dataset (multiple datasets not supported).
* **datasets** - an exact, case-sensitive match of one or more datasets.  All datasets must be present.
* **dataset_type** - an exact, case-sensitive match of one or more [dataset types](../datasets_and_types.html#Dataset-types).  All dataset types must be present.

#### Regex systems between modes
Regex searches can be made in remote mode by using special [Django queries](../useful_alyx_queries.html#regex), for example,
```
eids = one.search(django='subject__nickname__regex,^FD_04$', query_type='remote')
```
Regular expression syntax is different between modes, however: in remote mode the regex is parsed by a [PostgreSQL](https://www.postgresql.org/docs/current/functions-matching.html)
database, while in other modes it is done using Python's [re.search](https://docs.python.org/3/library/re.html#regular-expression-syntax).

## Searching insertions

A session may contain multiple insertions recording different brain areas.
To find data associated with a specific brain area, it is useful to search by insertion instead of
by session.

The `OneAlyx.search_insertions` method takes similar arguments to the remote search method,
and returns a list of probe UUIDs (pIDs), which can be interconverted with session IDs (eIDs) for
[loading data](../one_load/one_load.html#Load-spike-times-from-a-probe-UUID).

<div class="alert alert-info">
Note.

The search_insertions method is only available in [remote mode](../one_modes).
</div>

In [ ]:
one.search_terms('remote', 'insertions')
pids = one.search_insertions(atlas_acronym=['STR', 'CA3'], query_type='remote')
